# Выбор локации для скважины

Мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Требуется построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
#Импортируем требуемые для выполнения проекта библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import math

In [2]:
#Помещаем датасеты в переменные и выводим первые пять строк для ознакомления 
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')
print(data_1.head())
print(data_2.head())
print(data_3.head())



      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


In [3]:
#Посмотрим общую информацию о датасетах
print(data_1.info())
print(data_2.info())
print(data_3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

In [4]:
#Посмотрим описательные статистики, квантили
print(data_1.describe())
print(data_2.describe())
print(data_3.describe())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%       

Посмотрим как коррелируют друг с другом данные внутри датасетов

In [5]:
print(data_1.drop(['id'], axis=1).corr())

               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000


In [6]:
print(data_2.drop(['id'], axis=1).corr())

               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000


In [7]:
print(data_3.drop(['id'], axis=1).corr())

               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000


**Промежуточный вывод:**

Пропусков нет, размеры датасетов одинаковы, есть некоторые корреляции между столбцами датасетов, но поскольку данные синтетические нас это особо волновать не должно. В общем данные готовы, можно начинать в ML!!!

## Обучение и проверка модели

In [8]:
#Создадим список из датасетов для последующей передачи его в цикл
df_list = [data_1,data_2,data_3]

Поделим данные, обучим ЛинейнуюРегрессию и сделаем предсказания на для каждого датасета

In [9]:
#Объявим переменные для последующего сохранения в них целевых признаков и предстказаний
targets = []
predictions = []

#пройдемся цикклом по списку датасетов
for index, data in enumerate(df_list):
    
    #Складываем фичи в переменную
    features = data.drop(['id','product'], axis=1)
    
    #Кладем таргет
    target = data['product']
    
    #Делим датасет на обучающую и валидационную выборки
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size = .25, random_state=1234)
    
    #Сразу сохраняем таргет
    targets.append(target_valid)
    
    #Кладем алгоритм в переменную
    model = LinearRegression()
    
    #Обучаем
    model.fit(features_train, target_train)
    
    #Предсказываем на валидационных фичах
    predicted_valid = model.predict(features_valid)
    
    #Сохраняем предсказания
    predictions.append(predicted_valid)
    
    #Считаем среднеквадратическую ошибку
    rmse = mean_squared_error(target_valid, predicted_valid)**0.5
    
    #Выводим интересующие нас метрики в печать
    print(f'Средний запас предсказанного сырья в {index} регионе  - {predicted_valid.mean()}, rmse - {rmse}')

#Переформатируем переменные target и predictions в dataframe, транспонируем чтобы было привычней и меняем названия столбцов для удобства
targets = pd.DataFrame(targets).T.reset_index(drop=True)
targets.columns = [0,1,2]
predictions = pd.DataFrame(predictions).T

Средний запас предсказанного сырья в 0 регионе  - 92.43045952558441, rmse - 37.562394183637785
Средний запас предсказанного сырья в 1 регионе  - 68.80248249798366, rmse - 0.8939344738117703
Средний запас предсказанного сырья в 2 регионе  - 94.91573817913459, rmse - 40.10318099258418


In [10]:
#Выведем для проверки первые строки получившихся таблиц 
targets.head()

,0,1,2
0,82.940945,53.906522,77.366205
1,43.963568,110.992147,40.588032
2,125.408598,0.000000,57.820983
3,98.067233,137.945408,89.209380
4,64.533441,26.953261,120.450829


In [11]:
predictions.head()

,0,1,2
0,82.780945,53.253264,72.536223
1,41.891741,109.796493,66.579868
2,137.356107,-0.244098,80.751463
3,105.695487,137.133124,97.802814
4,99.545704,28.252534,113.885918


**2-ой промежуточный вывод:**

На трэйн-валид разбили, обучили/спрогнозировали, таргет и предсказания засэйвили, средний запас предсказанного сырья и rmse посчитали. Касательно rmse - e 0-го и 2-го датасета присудствует ощутимая СКО, скорей всего это означает то, что в данных имеются выбросы

## Подготовка к расчёту прибыли

Запишем основные величины в свои переменные и посчитаем **достаточный объём сырья для безубыточной разработки новой скважины** и сравним со средним запасом в каждой скважине.

In [12]:
#Стоимость тысячи баррелей
THOUSAND_BARREL_COST = 450000

#Общий бюджет на разработку
BUDGET= 10e9

#Количество скважин планируемых к разработке
POINTS = 200

#Достаточный объём сырья для безубыточной разработки новой скважины
QUANTITY_OF_PRODUCT_MIN =(BUDGET/THOUSAND_BARREL_COST)/POINTS


Теперь выведем и сравним средние запасы в каждом регионе и достаточный для безубыточной разработки объем сырья

In [13]:
print(QUANTITY_OF_PRODUCT_MIN)
print(data_1['product'].mean())
print(data_2['product'].mean())
print(data_3['product'].mean())


111.11111111111111
92.50000000000001
68.82500000000002
95.00000000000004


Да, без отбора самых прибыльных месторождений здесь никак.

Напишем функцию для подсчета суммарной прибыли с 200 самых прибыльных скважин

In [14]:
def revenue(predictions:pd.Series, targets:pd.Series, cost:int, quantity:int):
    #Ранжируем по убываню предсказанные объемы сырья
    preds_sorted = predictions.sort_values(ascending=False)
    
    #Теперь отберем месторождения из реальных данных с теми же индексами что и у 200 самых прибыльных предсказаний
    selected = targets[preds_sorted.index][:quantity]
    
    #Возвращаем суммарную прибыль с 200 самых прибыльных месторождений
    return (selected.sum()*cost)

**3-ий промежуточный вывод:** 

Без ранжирования по прибыльности в среднем запасы месторождений регионов убыточны.
Теперь отберем потенциально лучшие месторождения(200) и найдем их суммарную прибыль с заданной(95%) вероятностью

## Расчёт прибыли и рисков 

In [15]:
state = np.random.RandomState(12345)


#Напишем функцию в которой при помощи бутстрэпа найдем распределение прибыли, 95% доверительный интервал и риск не получить прибыль
def bootstrap_rev(predictions, targets, cost, points):
    
    #Создадим переменную куда будем складывать полученные бутстрапом прибыли
    values= []
    
    #Переменная для подсчета отрицательных значений прибыли
    negative_values = 0
    
    #Создадим тысячу подвыборок
    for i in range(1000):
        
        #Из каждой будет отобрано 500 случайных точек методом "с заменой"
        target_subsample = targets.sample(
            500, 
            random_state=state, 
            replace=True)
        
        #Выберем 500 месторождений с теми же индексами что и в target_subsample
        predictions_subsample = predictions[target_subsample.index]
        
        #Подсчитаем для 200 самых прибыльных из них суммарную выручку
        rev = revenue(predictions_subsample, target_subsample, cost, points)
        
        #Высчитываем прибыль и добавляем её в список
        values.append(rev - BUDGET)
        
        #Счетчик отрицательных прибылей
        if (rev - BUDGET) <= 0:
            negative_values +=1
    
    #Переформатируем с листа на серию 
    values = pd.Series(values)
    
    #Подсчитаем. доверительный интервал
    low_treshold = values.quantile(.025)
    high_treshold = values.quantile(.975)
    
    #Подсчитаем среднюю прибыль
    mean = values.mean()
    
    #Выведем в печать искомые переменные
    print(f'Средняя выручка -  Средняя прибыль - {mean}, 95% доверительный интервал- ({low_treshold} - {high_treshold}), риск не получить прибыль - {negative_values/values.count()}')
    print((values <0).mean())

In [16]:
bootstrap_rev(predictions[0], targets[0], THOUSAND_BARREL_COST , POINTS)
bootstrap_rev(predictions[1], targets[1], THOUSAND_BARREL_COST , POINTS)
bootstrap_rev(predictions[2], targets[2], THOUSAND_BARREL_COST , POINTS)

Средняя выручка -  Средняя прибыль - 411854134.6693436, 95% доверительный интервал- (-153036113.10503387 - 909822726.6300966), риск не получить прибыль - 0.065
0.065
Средняя выручка -  Средняя прибыль - 475927344.6527976, 95% доверительный интервал- (74247290.95463678 - 906509048.4226353), риск не получить прибыль - 0.013
0.013
Средняя выручка -  Средняя прибыль - 382102491.8959889, 95% доверительный интервал- (-183916236.36915794 - 946554521.4354271), риск не получить прибыль - 0.097
0.097


**Общий вывод:**

Исходя из полученных данных с 95%-ной вероятностью можно заявить следующее: 
    Регион №2 обладает близкой к 100% вероятностью не получить убытки, в отличии от регионов №1 и №3, которые вероятнее всего принесут убытки. Рекомендуем регион №2.